In [1]:
import tensorflow as tf
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"

In [2]:
from myUtils.utils import CifarData

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data)
print(batch_labels)

(10000, 3072)
(10000,)
(2000, 3072)
(2000,)
[[ 0.80392157  0.78039216  0.77254902 ...  0.74117647  0.71764706
   0.70196078]
 [ 0.20784314  0.18431373  0.18431373 ...  0.6         0.63137255
   0.6627451 ]
 [ 0.68627451  0.70980392  0.7254902  ... -0.60784314 -0.62352941
  -0.63921569]
 ...
 [-0.23137255 -0.17647059 -0.11372549 ...  0.39607843  0.30980392
   0.31764706]
 [ 0.81960784  0.80392157  0.80392157 ...  0.45098039  0.43529412
   0.42745098]
 [ 0.96078431  0.99215686  0.97647059 ...  0.96862745  0.98431373
   0.95294118]]
[1 0 0 1 0 1 0 0 0 0]


In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

w = tf.get_variable('w', [x.get_shape()[-1], 1], initializer=tf.random_normal_initializer(0, 1))
b = tf.get_variable('b', [1], initializer=tf.constant_initializer(0.0))

y_ = tf.matmul(x, w) + b
p_y_1 = tf.nn.sigmoid(y_)
y_reshaped = tf.reshape(y, (-1, 1))
y_reshaped_float = tf.cast(y_reshaped, tf.float32)

loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))
predict = p_y_1 > 0.5
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshaped)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x: batch_data, y: batch_labels})
        if (i + 1) % 500 == 0 :
            print("[Train] Step: %d, loss: %4.5f, accuracy: %4.5f" % (i + 1, loss_val, accu_val))
        if (i + 1) % 5000 == 0 :
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps) :
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print("[Test] Step: %d, accuracy: %4.5f" % (i + 1, test_acc))

[Train] Step: 500, loss: 0.24839, accuracy: 0.75000
[Train] Step: 1000, loss: 0.30090, accuracy: 0.70000
[Train] Step: 1500, loss: 0.19965, accuracy: 0.80000
[Train] Step: 2000, loss: 0.21005, accuracy: 0.80000
[Train] Step: 2500, loss: 0.05270, accuracy: 0.95000
[Train] Step: 3000, loss: 0.18969, accuracy: 0.80000
[Train] Step: 3500, loss: 0.24777, accuracy: 0.75000
[Train] Step: 4000, loss: 0.20107, accuracy: 0.80000
[Train] Step: 4500, loss: 0.19817, accuracy: 0.80000
[Train] Step: 5000, loss: 0.21284, accuracy: 0.75000
(2000, 3072)
(2000,)
[Test] Step: 5000, accuracy: 0.79800
[Train] Step: 5500, loss: 0.19248, accuracy: 0.80000
[Train] Step: 6000, loss: 0.14915, accuracy: 0.85000
[Train] Step: 6500, loss: 0.11594, accuracy: 0.85000
[Train] Step: 7000, loss: 0.05000, accuracy: 0.95000
[Train] Step: 7500, loss: 0.15008, accuracy: 0.85000
[Train] Step: 8000, loss: 0.24984, accuracy: 0.75000
[Train] Step: 8500, loss: 0.20985, accuracy: 0.80000
[Train] Step: 9000, loss: 0.10000, accurac

[Train] Step: 69500, loss: 0.10000, accuracy: 0.90000
[Train] Step: 70000, loss: 0.10207, accuracy: 0.90000
(2000, 3072)
(2000,)
[Test] Step: 70000, accuracy: 0.82100
[Train] Step: 70500, loss: 0.15081, accuracy: 0.85000
[Train] Step: 71000, loss: 0.10040, accuracy: 0.90000
[Train] Step: 71500, loss: 0.10004, accuracy: 0.90000
[Train] Step: 72000, loss: 0.10001, accuracy: 0.90000
[Train] Step: 72500, loss: 0.10020, accuracy: 0.90000
[Train] Step: 73000, loss: 0.00023, accuracy: 1.00000
[Train] Step: 73500, loss: 0.15022, accuracy: 0.85000
[Train] Step: 74000, loss: 0.30000, accuracy: 0.70000
[Train] Step: 74500, loss: 0.05915, accuracy: 0.95000
[Train] Step: 75000, loss: 0.10002, accuracy: 0.90000
(2000, 3072)
(2000,)
[Test] Step: 75000, accuracy: 0.82000
[Train] Step: 75500, loss: 0.15029, accuracy: 0.85000
[Train] Step: 76000, loss: 0.15132, accuracy: 0.85000
[Train] Step: 76500, loss: 0.10000, accuracy: 0.90000
[Train] Step: 77000, loss: 0.15000, accuracy: 0.85000
[Train] Step: 7750